<a href="https://colab.research.google.com/github/lg550055/pe-backend/blob/main/yhooFin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import json
import requests
headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) (KHTML, like Gecko) Chrome/102.0.5005.63'}
ticker = 'WMT'

In [19]:
url = 'https://finance.yahoo.com/quote/'+ ticker + '/analysis?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]
# only the data["QuoteSummaryStore"]['earningsTrend']['trend'] object seems to have relevant info

In [ ]:
# field = [t for t in data["context"]["dispatcher"]["stores"]["QuoteSummaryStore"]["earningsTrend"]["trend"] if t["period"] == "+5y" ][0]
f = [t for t in data["QuoteSummaryStore"]["earningsTrend"]["trend"] if t["period"] in ['0y','+1y']]
num_analysts = f[0]['earningsEstimate']['numberOfAnalysts']['fmt']
fwd_eps = f[0]['earningsEstimate']['avg']['fmt']
fwd_rev = f[0]['revenueEstimate']['avg']['raw']
fwd_rev_g = f[0]['revenueEstimate']['growth']['raw']
fwd2_eps = f[1]['earningsEstimate']['avg']['fmt']
fwd2_rev = f[1]['revenueEstimate']['avg']['fmt']
fwd2_rev_g = f[1]['revenueEstimate']['growth']['fmt']
print(num_analysts, fwd_eps, fwd_rev, fwd_rev_g, fwd2_eps, fwd2_rev, fwd2_rev_g)
# print("Next 5 Years (per annum) : " + field["growth"]["fmt"])

35 6.41 595703000000 0.04 6.96 616.08B 3.40%


In [ ]:
# url for is, bs, cf -but not all items, e.g. q dil shrs or net debt
url = 'https://finance.yahoo.com/quote/'+ ticker + '/financials?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]

In [ ]:
# rev = data["QuoteSummaryStore"]['incomeStatementHistory']['incomeStatementHistory'][0]['totalRevenue']['fmt']
# eop = data["QuoteSummaryStore"]['incomeStatementHistory']['incomeStatementHistory'][0]['endDate']['fmt']
# bseop = data["QuoteSummaryStore"]['balanceSheetHistory']['balanceSheetStatements'][0]['endDate']['fmt']
# dshrs = data["QuoteTimeSeriesStore"]['timeSeries']['annualDilutedAverageShares'][-1]['reportedValue']['fmt']
# data["QuoteTimeSeriesStore"]['timeSeries']['trailingNormalizedEBITDA'][-1]['reportedValue']['fmt']
# cfeop = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][0]['endDate']['fmt']

s = data["QuoteTimeSeriesStore"]['meta']['symbol']
p = data["QuoteSummaryStore"]['price']['regularMarketPrice']['fmt']
mcap = data["QuoteSummaryStore"]['price']['marketCap']['fmt']
eop = data["QuoteTimeSeriesStore"]['timeSeries']['annualTotalRevenue'][-1]['asOfDate']
rev = data["QuoteTimeSeriesStore"]['timeSeries']['annualTotalRevenue'][-1]['reportedValue']['raw'] / 1e9
ebit = data["QuoteTimeSeriesStore"]['timeSeries']['annualEBIT'][-1]['reportedValue']['raw'] / 1e9
opi = data["QuoteTimeSeriesStore"]['timeSeries']['annualOperatingIncome'][-1]['reportedValue']['raw'] / 1e9
ebitda = data["QuoteTimeSeriesStore"]['timeSeries']['annualNormalizedEBITDA'][-1]['reportedValue']['raw'] / 1e9
capex = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][0]['capitalExpenditures']['raw'] / -1e9
cfo = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][0]['totalCashFromOperatingActivities']['raw'] / 1e9
eps = data["QuoteTimeSeriesStore"]['timeSeries']['annualDilutedEPS'][-1]['reportedValue']['fmt']
print(s, p, mcap, eop, rev, opi, ebit, ebitda, ebitda-capex, cfo, eps, round(opi/rev*100,1), round(ebit/rev*100,1), round(ebitda/rev*100,1))

WMT 122.63 336.15B 2022-01-31 572.754 25.942 20.69 33.758 20.652 24.181 4.87 4.5 3.6 5.9


In [15]:
# url for netDebt
url = 'https://finance.yahoo.com/quote/'+ ticker + '/balance-sheet?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]
# data.keys()

In [17]:
data["QuoteTimeSeriesStore"]['timeSeries']['annualNetDebt'][-1]

{'asOfDate': '2022-01-31',
 'currencyCode': 'USD',
 'dataId': 23531,
 'periodType': '12M',
 'reportedValue': {'fmt': '23.32B', 'raw': 23317000000}}

## Selenium

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
d = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
ss = input('Stock symbol?')
d.get(f'https://finance.yahoo.com/quote/{ss}/analysis?p={ss}')
main = d.find_element_by_id('Main')
tds = main.find_elements_by_tag_name('td')
if tds:
  data = [e.text for e in tds]
  print(data[0],data[4],data[5],data[8],data[9],data[30],data[33],data[34],data[50],data[53],data[54])
else:
  print('no data')
d.quit()